In [4]:
import pandas as pd
import nltk
import os
from nltk.corpus import stopwords
import string
import re
from nltk.tokenize import word_tokenize
import pymorphy2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Импорт библиотек**

In [5]:
stopwords = nltk.corpus.stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()
wn = nltk.WordNetLemmatizer()
def remove_punctuation(text):
    text = str(text)
    return ''.join([ch for ch in text if ch not in
    string.punctuation])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_notalpha(text):
    return ''.join(i if i.isalpha() else ' ' for i in text)


def remove_space(text):
    return re.sub(r'\s+',' ', text, flags=re.I)


def tokenize(text):
    t = word_tokenize(text)
    return [token for token in t if token not in stopwords]


def lemmatize(text):
    res = list()
    for word in text:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

**Функции для обработки текста**

In [7]:
def textProcessing(df):
    text_list = []
    for i in df['title']:
        if i is not None:
            i = remove_punctuation(i)
            i = remove_numbers(i)
            i = remove_notalpha(i)
            i = remove_space(i)
            i = tokenize(i)
            i = lemmatize(i)
            text_list.append(i)
    return text_list


def urlProcessing(df):
    url_list = []
    keywords = {
        "porn", "sex", "adult", "video", "cam", "cum", "blowjob", "xxx", "x", "ero", "anal"
        "boobs", "onlyfans", "hub", "model", "fap", "girl", "lesbian", "hot", "horny", "teen", 
        "milf", "asian", "hd", "pussy", "dick", "fuck", "nude", "couple"
    }
    for url in df['url']:
        found_keywords = []
        if isinstance(url, str):
            url_lower = url.lower()
            for kw in keywords:
                if kw in url_lower:
                    found_keywords.append(kw)
        url_list.append(" ".join(found_keywords))
    return url_list

**Обработка текста и url**

In [8]:
df = pd.read_csv("/kaggle/input/ml-2025-spring-porn-detection/train.csv", encoding='utf-8')
test_df = pd.read_csv("/kaggle/input/ml-2025-spring-porn-detection/test.csv", encoding='utf-8')

**Загрузка данных**

In [16]:
def formFeautures(text_list, url_list):
    X = []
    for text, url in zip(text_list, url_list):
        X.append(str(text) + " " + str(url))
    return X
def trainModel(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    
    model = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 1))),
        ('logreg', LogisticRegression(solver='saga', max_iter=1000, C=10))
    ])
    
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("\nРезультат на тестовых данных:")
    print(classification_report(y_test, y_pred))
    
    return model
text_list, url_list = textProcessing(df), urlProcessing(df)
X = formFeautures(text_list, url_list)
y = df['label']
model = trainModel(X, y)


Результат на тестовых данных:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     35582
           1       0.99      0.95      0.97      5011

    accuracy                           0.99     40593
   macro avg       0.99      0.97      0.98     40593
weighted avg       0.99      0.99      0.99     40593



**Формирование признаков и обучение модели**

In [14]:
test_text_list, test_url_list = textProcessing(test_df), urlProcessing(test_df)
test_X = formFeautures(test_text_list, test_url_list)
y_predict = model.predict(test_X)
result_df = pd.DataFrame({'ID': test_df['ID'],
                          'label': y_predict})
result_df.to_csv('submission.csv', index=False)

**Создание submission файла**